### How to chat with GPT using OpenAI API

In [1]:
import os
import openai
from retrying import retry
from IPython.display import Markdown,display
print ("openai version:[%s]"%(openai.__version__))

openai version:[0.27.8]


### Locate where your key is

In [2]:
key_path = '../key/rilab_key.txt'
print ('key_path:[%s]'%(key_path))

key_path:[../key/rilab_key.txt]


### Use key

In [3]:
with open(key_path, 'r') as f: OPENAI_API_KEY = f.read()
openai.api_key = OPENAI_API_KEY

### Query function

In [4]:
@retry(stop_max_attempt_number=5,
       wait_exponential_multiplier=1000,
       wait_exponential_max=10000)
def query_gpt(messages:list,gpt_model='gpt-4'):
    """
        gpt_model: 'gpt-3.5-turbo' / 'gpt-4'
    """
    # Call the OpenAI API
    response     = openai.ChatCompletion.create(
        model    = gpt_model, 
        messages = messages
    )
    # Extract the response content and status code
    content     = response["choices"][0]["message"]["content"]
    status_code = response["choices"][0]["finish_reason"]
    return content,status_code,response
print ("Ready.")

Ready.


`messages`, an input to GPT, is basically a list where each item is a dictionary consists of `role` and `content`. A `role` can either be
* `system`: which defines the identity of the agent
* `user`: which states the input of a user
* `assistant`: which stores messages previously generated by the agents
More information can be found in [here](https://platform.openai.com/docs/guides/gpt/chat-completions-api).

In [5]:
role_msg = """너는 한국어로 대화에 능통한 언어모델이야"""
question = "LLM(Large Language Model)은 어떻게 작동하는지 설명해줘"
messages = [{"role": "system", "content": f'{role_msg}'},
            {"role": 'user', "content": f'{question}'}]
print (messages)

[{'role': 'system', 'content': '너는 한국어로 대화에 능통한 언어모델이야'}, {'role': 'user', 'content': 'LLM(Large Language Model)은 어떻게 작동하는지 설명해줘'}]


### Now let's use `GPT`

In [6]:
content,status_code,response = query_gpt(messages=messages)

In [7]:
print (content)

LLM(Large Language Model)은 인공지능이 텍스트를 이해하고 생성하는 능력을 가진 모델입니다. 그 중 가장 대표적인 것이 GPT-3 같은 트랜스포머 기반의 모델들이죠.

이 모델들은 대량의 텍스트 데이터로 학습하는데요. 학습 과정에서는 문장이나 문단, 또는 웹페이지나 책 등 다양한 길이의 텍스트 조각들이 입력되며, 그 다음에 올 텍스트를 예측하도록 학습됩니다. 

트랜스포머 기반의 모델들은 인코더와 디코더 구조를 가지고 있어서, 인코더가 입력 텍스트를 내부적인 상태로 변환하고, 디코더가 그 상태를 다시 텍스트로 변환하는 데 사용됩니다. 그리고 이런 변환 과정에서 어떤 단어 또는 문구가 다음에 나타날 확률이 가장 높은지를 모델화합니다.

모델이 이렇게 학습하면서 언어의 문법, 단어가 가진 의미, 문장 구조, 문맥 등 다양한 것들을 배우게 되는데, 이런 지식은 모델이 새로운 텍스트를 생성할 때 사용됩니다. 그리고 이것이 가능하게 하는 핵심 기술이 바로 딥러닝, 즉 깊은 신경망입니다.

마지막으로, 아무리 훌륭한 모델이라도 제 제약사항과 한계가 있기 때문에, LLM은 사용자의 의도를 완벽하게 이해하거나, 거짓 정보를 구별하는 능력 등을 가질 수는 없습니다.


In [8]:
print (status_code)

stop


In [9]:
print (response)

{
  "id": "chatcmpl-7alClH4rQIUrhnbj7qhQodHTTQqQ9",
  "object": "chat.completion",
  "created": 1688995619,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "LLM(Large Language Model)\uc740 \uc778\uacf5\uc9c0\ub2a5\uc774 \ud14d\uc2a4\ud2b8\ub97c \uc774\ud574\ud558\uace0 \uc0dd\uc131\ud558\ub294 \ub2a5\ub825\uc744 \uac00\uc9c4 \ubaa8\ub378\uc785\ub2c8\ub2e4. \uadf8 \uc911 \uac00\uc7a5 \ub300\ud45c\uc801\uc778 \uac83\uc774 GPT-3 \uac19\uc740 \ud2b8\ub79c\uc2a4\ud3ec\uba38 \uae30\ubc18\uc758 \ubaa8\ub378\ub4e4\uc774\uc8e0.\n\n\uc774 \ubaa8\ub378\ub4e4\uc740 \ub300\ub7c9\uc758 \ud14d\uc2a4\ud2b8 \ub370\uc774\ud130\ub85c \ud559\uc2b5\ud558\ub294\ub370\uc694. \ud559\uc2b5 \uacfc\uc815\uc5d0\uc11c\ub294 \ubb38\uc7a5\uc774\ub098 \ubb38\ub2e8, \ub610\ub294 \uc6f9\ud398\uc774\uc9c0\ub098 \ucc45 \ub4f1 \ub2e4\uc591\ud55c \uae38\uc774\uc758 \ud14d\uc2a4\ud2b8 \uc870\uac01\ub4e4\uc774 \uc785\ub825\ub418\uba70, \uadf8 

### Helper Class for implementing efficient chat with GPT

In [10]:
def printmd(string):
    display(Markdown(string))
class GPTchatClass():
    def __init__(self,
                 gpt_model = 'gpt-4',
                 role_msg  = 'Your are a helpful assistant.',
                 VERBOSE   = True
                ):
        self.gpt_model     = gpt_model
        self.messages      = [{'role':'system','content':f'{role_msg}'}]
        self.init_messages = [{'role':'system','content':f'{role_msg}'}]
        self.VERBOSE       = VERBOSE
        self.response      = None
        if self.VERBOSE:
            print ("Chat agent using [%s] initialized with the follow role:[%s]"%
                   (self.gpt_model,role_msg))
    
    def _add_message(self,role='assistant',content=''):
        """
            role: 'assistant' / 'user'
        """
        self.messages.append({'role':role, 'content':content})
        
    def _get_response_content(self):
        if self.response:
            return self.response['choices'][0]['message']['content']
        else:
            return None
        
    def _get_response_status(self):
        if self.response:
            return self.response['choices'][0]['message']['finish_reason']
        else:
            return None
    
    def chat(self,user_msg='hi',
             PRINT_USER_MSG=True,PRINT_GPT_OUTPUT=True,
             RESET_CHAT=False,RETURN_RESPONSE=True):
        self._add_message(role='user',content=user_msg)
        self.response = openai.ChatCompletion.create(
            model    = self.gpt_model,
            messages = self.messages
        )
        # Backup response for continous chatting
        self._add_message(role='assistant',content=self._get_response_content())
        if PRINT_USER_MSG:
            print("[USER_MSG]")
            printmd(user_msg)
        if PRINT_GPT_OUTPUT:
            print("[GPT_OUTPUT]")
            printmd(self._get_response_content())
        # Reset
        if RESET_CHAT:
            self.messages = self.init_messages
        # Return
        if RETURN_RESPONSE:
            return self._get_response_content()
print ("Ready.")              

Ready.


### Now let's chat

In [11]:
GPT = GPTchatClass(gpt_model='gpt-4',role_msg  = 'Your are a helpful assistant.')
PRINT_USER_MSG   = True
PRINT_GPT_OUTPUT = True
RESET_CHAT       = False
RETURN_RESPONSE  = False
GPT.chat(user_msg='Who is the current president of Korea?',
         PRINT_USER_MSG=PRINT_USER_MSG,PRINT_GPT_OUTPUT=PRINT_GPT_OUTPUT,
         RESET_CHAT=RESET_CHAT,RETURN_RESPONSE=RETURN_RESPONSE)
GPT.chat(user_msg='Are you sure? I think you are outdated.',
         PRINT_USER_MSG=PRINT_USER_MSG,PRINT_GPT_OUTPUT=PRINT_GPT_OUTPUT,
         RESET_CHAT=RESET_CHAT,RETURN_RESPONSE=RETURN_RESPONSE)
GPT.chat(user_msg='Where can I get the latest information?',
         PRINT_USER_MSG=PRINT_USER_MSG,PRINT_GPT_OUTPUT=PRINT_GPT_OUTPUT,
         RESET_CHAT=RESET_CHAT,RETURN_RESPONSE=RETURN_RESPONSE)

Chat agent using [gpt-4] initialized with the follow role:[Your are a helpful assistant.]
[USER_MSG]


Who is the current president of Korea?

[GPT_OUTPUT]


As of my last update, the current president of South Korea is Moon Jae-in. He has been in office since May 10, 2017. Please check the latest sources for the most recent information.

[USER_MSG]


...

I'm sorry for any confusion. As of March 10, 2023, the President of South Korea is Yoon Suk-yeol.

[USER_MSG]


Where can I get the latest information?

[GPT_OUTPUT]


You can get the latest information about current world leaders from reliable news sources such as:

1. International News Websites: Websites like BBC, CNN, or Al Jazeera regularly update their content to reflect recent changes in leadership.
2. Official Government Websites: In this instance, the official government website of South Korea.
3. Google News: Google News aggregates news from many different sources and is usually up-to-date.
4. Social Media: Social media platforms like Twitter often have real-time updates for major world events. 
   
Remember to ensure that the information source is reliable before taking it as fact.

### Chat with resetting everytime

In [12]:
GPT = GPTchatClass(gpt_model='gpt-4',role_msg  = 'Your are a helpful assistant.')
PRINT_USER_MSG   = True
PRINT_GPT_OUTPUT = True
RESET_CHAT       = True
RETURN_RESPONSE  = False
GPT.chat(user_msg='Who is the current president of Korea?',
         PRINT_USER_MSG=PRINT_USER_MSG,PRINT_GPT_OUTPUT=PRINT_GPT_OUTPUT,
         RESET_CHAT=RESET_CHAT,RETURN_RESPONSE=RETURN_RESPONSE)
GPT.chat(user_msg='Are you sure? I think you are outdated.',
         PRINT_USER_MSG=PRINT_USER_MSG,PRINT_GPT_OUTPUT=PRINT_GPT_OUTPUT,
         RESET_CHAT=RESET_CHAT,RETURN_RESPONSE=RETURN_RESPONSE)
GPT.chat(user_msg='Where can I get the latest information?',
         PRINT_USER_MSG=PRINT_USER_MSG,PRINT_GPT_OUTPUT=PRINT_GPT_OUTPUT,
         RESET_CHAT=RESET_CHAT,RETURN_RESPONSE=RETURN_RESPONSE)

Chat agent using [gpt-4] initialized with the follow role:[Your are a helpful assistant.]
[USER_MSG]


Who is the current president of Korea?

[GPT_OUTPUT]


The current president of South Korea, as of 2022, is Moon Jae-in. He has been in office since May 10, 2017. However, please verify from a up-to-date source, as this information might change over time.

[USER_MSG]


...

As an AI, I'm always connected to the internet and periodically updated. My main goal is to help and provide the most efficient, updated information and assistance for any questions you have. How may I assist you further?

[USER_MSG]


Where can I get the latest information?

[GPT_OUTPUT]


The source of the latest information depends on the type of information you're looking for. Here are a few possibilities:

1. News: Websites like CNN, BBC, and Al Jazeera, among others. They also have apps you can download on your phone.
2. Technology: Websites like TechCrunch, CNET, Wired, or Gizmodo provide the latest tech news and updates.
3. Academic: Websites like Google Scholar, JSTOR, and EBSCOhost have huge databases full of academic articles.
4. Business/Finance: Websites like Bloomberg, Financial Times, and The Wall Street Journal are major sources for business and finance news.
5. Entertainment: Websites like The Hollywood Reporter, Billboard, or IMDB have the latest in entertainment news.

Please specify the area you're interested in for more detailed advice!